In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
from datetime import datetime
from tqdm import *

In [20]:
# IMPORT DATA

In [21]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [22]:
sess = tf.InteractiveSession()

In [23]:
# BUILDING THE MODEL

In [24]:
# tf.placeholder: a value that we will input when we ask tf to run a computation
x = tf.placeholder(tf.float32, shape=[None, 784]) # Represents input dimensions
y_ = tf.placeholder(tf.float32, shape=[None, 10]) # Represents output dimensions

# tf.Variable: value that lives in the computational graph 
W = tf.Variable(tf.zeros([784, 10])) # [Input dims, output dims] -> Weights
b = tf.Variable(tf.zeros([10])) # output classes/ dims -> biases

# Before tf.Variables can be used in tf.Session(), they must be initialized: ERROR
# variable_initializers(local_variables())
sess.run(tf.global_variables_initializer())

# Implement Logistic Regression Formula
y = tf.matmul(x, W) + b

# Implement loss function -> cross_entropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

# TRAIN THE MODEL
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy) # Learning Rate - Loss Function

for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
# We load a batch of 100 in each training iteration
# We then use the train_step operation, using the feed_dict argument we replace the placeholder tensors 
#  x & y_ with their respective training examples

# EVALUATE THE MODEL
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9193


In [25]:
# BUILDING A MULTILAYER CONVOLUTIONAL NETWORK

In [28]:
# Weight Initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 100, training accuracy 0.08
step 200, training accuracy 0.08
step 300, training accuracy 0.14
step 400, training accuracy 0.12
step 500, training accuracy 0.16
step 600, training accuracy 0.12
step 700, training accuracy 0.32
step 800, training accuracy 0.16
step 900, training accuracy 0.28
step 1000, training accuracy 0.32
step 1100, training accuracy 0.24
step 1200, training accuracy 0.14
step 1300, training accuracy 0.12
step 1400, training accuracy 0.22
step 1500, training accuracy 0.4
step 1600, training accuracy 0.44
step 1700, training accuracy 0.42
step 1800, training accuracy 0.44
step 1900, training accuracy 0.44
step 2000, training accuracy 0.46
step 2100, training accuracy 0.44
step 2200, training accuracy 0.46
step 2300, training accuracy 0.52
step 2400, training accuracy 0.4
step 2500, training accuracy 0.4
step 2600, training accuracy 0.6
step 2700, training accuracy 0.52
step 2800, training accuracy 0.64
step 2900, training accuracy 0

In [ ]:
# TensorBoard: tool that makes tf easier to understand, debug, and optimize.
# Uses:
#    Visualize tf graph
#    Plot quantitative metrics about the execution of the computational graph
#    Show additional data like images passed through a conv net

In [15]:
'''
Graph and Loss visualization using Tensorboard.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = '/tmp/tensorflow_logs/example'

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 1.182138980
Epoch: 0002 cost= 0.664811819
Epoch: 0003 cost= 0.552594316
Epoch: 0004 cost= 0.498513847
Epoch: 0005 cost= 0.465453073
Epoch: 0006 cost= 0.442464822
Epoch: 0007 cost= 0.425458328
Epoch: 0008 cost= 0.412161023
Epoch: 0009 cost= 0.401298949
Epoch: 0010 cost= 0.392337930
Epoch: 0011 cost= 0.384692998
Epoch: 0012 cost= 0.378103420
Epoch: 0013 cost= 0.372347895
Epoch: 0014 cost= 0.367254965
Epoch: 0015 cost= 0.362716609
Epoch: 0016 cost= 0.358617316
Epoch: 0017 cost= 0.354830579
Epoch: 0018 cost= 0.351468265
Epoch: 0019 cost= 0.348293769
Epoch: 0020 cost= 0.345450685
Epoch: 0021 cost= 0.342717626
Epoch: 0022 cost= 0.340235036
Epoch: 0023 cost= 0.337923101
Epoch: 0024 cost= 0.335736258
Epoch: 0025 cost